In [1]:
%%capture
!pip install xstac

In [5]:
# List all zarr data in test-data
import s3fs
import sys; sys.path.append('..');
import helpers.eodc_hub_role as eodc_hub_role

In [6]:
credentials = eodc_hub_role.fetch_and_set_credentials()
bucket = 'nasa-eodc-data-store'
fake_data_dir = 'fake_data'
s3_fs = s3fs.S3FileSystem(
    key=credentials['AccessKeyId'],
    secret=credentials['SecretAccessKey'],
    token=credentials['SessionToken'], 
    anon=False
)

In [10]:
data_path = 'test-data'
directories = s3_fs.walk(f'{bucket}/{data_path}', maxdepth=3)
for directory in directories:
    print(directory)

('nasa-eodc-data-store/test-data', ['cmip6-kerchunk', 'cmip6-zarr', 'fake-data'], [''])
('nasa-eodc-data-store/test-data/cmip6-kerchunk', [], ['combined_CMIP6_daily_GISS-E2-1-G_tas_kerchunk.json'])
('nasa-eodc-data-store/test-data/cmip6-zarr', ['365_262_262', '600_1440_1', '600_1440_29'], [])
('nasa-eodc-data-store/test-data/cmip6-zarr/365_262_262', ['CMIP6_daily_GISS-E2-1-G_tas.zarr'], [])
('nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_1', ['CMIP6_daily_GISS-E2-1-G_tas.zarr'], [])
('nasa-eodc-data-store/test-data/cmip6-zarr/600_1440_29', ['CMIP6_daily_GISS-E2-1-G_tas.zarr'], [])
('nasa-eodc-data-store/test-data/fake-data', ['single_chunk', 'with_chunks'], [])
('nasa-eodc-data-store/test-data/fake-data/single_chunk', ['store_lat1024_lon2048.zarr', 'store_lat1448_lon2896.zarr', 'store_lat2048_lon4096.zarr', 'store_lat2896_lon5792.zarr', 'store_lat4096_lon8192.zarr', 'store_lat512_lon1024.zarr', 'store_lat724_lon1448.zarr'], [])
('nasa-eodc-data-store/test-data/fake-data/with_chunk

In [40]:
from datetime import datetime
import pandas
import pystac
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import xstac
import sys; sys.path.append('..')
from titiler_xarray.titiler.xarray.reader import xarray_open_dataset, ZarrReader

def generate_stac(url: str, idstr: str = None):
    ds = xarray_open_dataset(url)
    # get extent
    spatial_extent_values = [ds.lon[0].values, ds.lat[0].values, ds.lon[-1].values, ds.lat[-1].values]
    spatial_extent = list(map(int, spatial_extent_values))
    # TODO(aimee): temporal extents are all over the place, probably want to pass this as an argument.
    # temporal_extent_values = [ds.time[0].values, ds.time[-1].values]
    # temporal_extent = [datetime.utcfromtimestamp(int(t)/1e9) for t in temporal_extent_values]

    if idstr:
        _id = idstr
    else:
        _id = url.split('/')[-1]
    zarr_asset = pystac.Asset(
        title='zarr',
        href=url,
        media_type='application/vnd+zarr',
        roles=['data'],
    )

    extent = pystac.Extent(
        spatial=pystac.SpatialExtent(bboxes=[spatial_extent]),
        temporal=pystac.TemporalExtent([[None, None]])
    )
   
    collection = pystac.Collection(
        id=_id,
        extent=extent,
        assets = {'zarr-s3': zarr_asset},
        description='for zarr testing',
        stac_extensions=['https://stac-extensions.github.io/datacube/v2.0.0/schema.json']
    )
    collection_template = collection.to_dict()
    collection = xstac.xarray_to_stac(
        ds,
        collection_template,
        temporal_dimension="time",
        x_dimension="lon",
        y_dimension="lat",
        # TODO: get this from attributes if possible
        reference_system="4326"
    )
    return collection

In [42]:
stac = generate_stac('s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr')
stac

id: power_901_monthly_meteorology_utc.zarr
description: for zarr testing
type: Collection
"cube:dimensions: {'time': {'extent': ['1981-01-31T00:00:00Z', '2021-12-31T00:00:00Z'], 'type': 'temporal'}, 'lon': {'axis': 'x', 'extent': [-180.0, 179.375], 'description': 'longitude', 'reference_system': 4326, 'type': 'spatial'}, 'lat': {'axis': 'y', 'extent': [-90.0, 90.0], 'description': 'latitude', 'reference_system': 4326, 'type': 'spatial'}}"
https://stac-extensions.github.io/datacube/v2.0.0/schema.json
rel: root
href: /collection.json
type: application/json
rel: self
href: /collection.json
type: application/json


In [41]:
url = 's3://cmip6-pds/CMIP6/CMIP/NASA-GISS/GISS-E2-1-G/historical/r2i1p1f1/Amon/tas/gn/v20180827/'
stac = generate_stac(url, idstr='cmip6-pds_GISS-E2-1-G_historical_tas')
stac

/Users/aimeebarciauskas/github/developmentseed/tile-benchmarking/profiling/../titiler_xarray/titiler/xarray/reader.py:42: UserWarning: Variable(s) referenced in cell_measures not in variables: ['areacella']
  return xarray.open_dataset(src_path, **xr_open_args)


id: cmip6-pds_GISS-E2-1-G_historical_tas
description: for zarr testing
type: Collection
"cube:dimensions: {'time': {'extent': ['1850-01-16T12:00:00Z', '2014-12-16T12:00:00Z'], 'description': 'time', 'type': 'temporal'}, 'lon': {'axis': 'x', 'extent': [1.25, 358.75], 'step': 2.5, 'description': 'longitude', 'reference_system': 4326, 'type': 'spatial'}, 'lat': {'axis': 'y', 'extent': [-89.0, 89.0], 'step': 2.0, 'description': 'latitude', 'reference_system': 4326, 'type': 'spatial'}}"
"cube:variables: {'height': {'type': 'auxiliary', 'description': 'height', 'dimensions': [], 'unit': 'm', 'attrs': {'axis': 'Z', 'long_name': 'height', 'positive': 'up', 'standard_name': 'height', 'units': 'm'}, 'shape': []}, 'lat_bnds': {'type': 'auxiliary', 'dimensions': ['lat', 'bnds'], 'attrs': {}, 'shape': [90, 2]}, 'lon_bnds': {'type': 'auxiliary', 'dimensions': ['lon', 'bnds'], 'attrs': {}, 'shape': [144, 2]}, 'tas': {'type': 'data', 'description': 'Near-Surface Air Temperature', 'dimensions': ['time', 'lat', 'lon'], 'unit': 'K', 'attrs': {'cell_methods': 'area: time: mean', 'comment': 'near-surface (usually, 2 meter) air temperature', 'long_name': 'Near-Surface Air Temperature', 'standard_name': 'air_temperature', 'units': 'K'}, 'shape': [1980, 90, 144]}, 'time_bnds': {'type': 'auxiliary', 'dimensions': ['time', 'bnds'], 'attrs': {}, 'shape': [1980, 2]}}"
https://stac-extensions.github.io/datacube/v2.0.0/schema.json
rel: root
href: /collection.json
type: application/json
rel: self
href: /collection.json
